In [1]:
import csv
import pandas as pd
import re
import requests
import os
from unidecode import unidecode
from datetime import datetime


# Ingest Data

In [2]:
lineups = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/New Data March 3/Match Lineups _ Goals Final Version - All Lineups.csv')
lineups = lineups.drop(columns={'Unnamed: 0'})

results = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/New Data March 3/Match Results - International Match Results.csv')
results = results.drop(columns={'Unnamed: 0'})

countrycodes = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/countries_and_codes.csv')

## Functions

In [3]:
def remove_accents(text):
    return unidecode(text)

def get_country_code(country):
    matching_rows = countrycodes.loc[countrycodes['Country'] == country, ' Code']
    return matching_rows.iloc[0] if not matching_rows.empty else None

### Add rows to countryCodes database

In [4]:
# Add a new column 'Accents Removed'
countrycodes['Accents Removed'] = countrycodes['Country'].apply(remove_accents)
countrycodes.at[48, 'Country'] = 'China, PR'


data = {'Country': ['British Virgin Islands'], ' Code': ['BVI'], 'Accents Removed': ['British Virgin Islands']}
one_row_dataframe = pd.DataFrame(data)
countrycodes = pd.concat([countrycodes, one_row_dataframe], ignore_index=True)

data = {'Country': ['North Ireland'], ' Code': ['NIR'], 'Accents Removed': ['North Ireland']}
one_row_dataframe = pd.DataFrame(data)
countrycodes = pd.concat([countrycodes, one_row_dataframe], ignore_index=True)




## WORKING on RESULTS

### Finding missing rows & fixing country names in results

#### Team 1

In [5]:
missing = []

for i in range(0, len(results)):
    
    if results.at[i, 'Team 1'] in countrycodes['Country'].values or results.at[i, 'Team 1'] in countrycodes['Accents Removed'].values:
        0==0
    else:
        if(results.at[i, 'Team 1'] in missing):
            0==0
        else:
            missing.append(results.at[i, 'Team 1'])
            print(i, results.at[i, 'Team 1'])


52 nan


#### Team 2

In [6]:
missing2 = []

for i in range(0, len(results)):
    
    if results.at[i, 'Team 2'] in countrycodes['Country'].values or results.at[i, 'Team 2'] in countrycodes['Accents Removed'].values:
        0==0
    else:
        if(results.at[i, 'Team 2'] in missing2):
            0==0
        else:
            missing2.append(results.at[i, 'Team 2'])
            print(i, results.at[i, 'Team 2'])


52 nan


### Adding Country ID to results

In [7]:
results = results.drop(52, axis=0).reset_index().drop(columns='index', axis=1)


In [8]:
def process_date(date):
    # Check if the date is NaN or not a string
    if pd.isna(date) or not isinstance(date, str):
        return date  # Return NaN or non-string values as they are
    elif ' ' in date:
        # Split the date and time
        date_parts = date.split(' ')
        # Keep only the date part
        return date_parts[0]
    else:
        return date

# Apply the function to the 'Date' column
results['Date'] = results['Date'].apply(process_date)


In [9]:
# Add new columns 'Team 1 Code' and 'Team 2 Code'
results['Team 1 Code'] = results['Team 1'].apply(get_country_code)
results['Team 2 Code'] = results['Team 2'].apply(get_country_code)

results = results.reset_index()
results = results.rename(columns={'index': 'Match ID'})


In [10]:
for i in range(0, len(results)):
    if(results.at[i, 'Team 1'] == 'Namibia'):
        results.at[i, 'Team 1 Code'] = 'NA'
    if(results.at[i, 'Team 2'] == 'Namibia'):
        results.at[i, 'Team 2 Code'] = 'NA'

### REsults - handle U-20 games

In [11]:
for i in range(0, len(results)):
    # Check if 'U-20' is present in the 'COMPETITION' column
    if(pd.isna(results.at[i,'Competition'])):
        0==0
    else:
        if 'U-20' in results.at[i,'Competition']:
            # Add 'U20' to the end of 'TEAM 1' and 'TEAM 2'
            results.at[i, 'Team 1'] += ' U-20'
            results.at[i, 'Team 2'] += ' U-20'

            #bit to change
            teams_names = results.at[i, 'SearchString'].split(' on ')[0]

            team_name_1 = teams_names.split(' vs ')[0]
            team_name_2 = teams_names.split(' vs ')[1]

            team_name_1 = team_name_1 + " U-20"
            team_name_2 = team_name_2 + " U-20"

            teams_names = team_name_1 + " vs " + team_name_2

            #bit to keep the same
            rest_of_searchstring = results.at[i, 'SearchString'].split(' on ')[1]

            #putting them together 
            results.at[i, 'SearchString'] = teams_names + " on "+ rest_of_searchstring 


## WORKING ON LINEUPS

### Finding missing rows & fixing country names in Lineups

In [12]:
for i in range(0, len(lineups)):
    if lineups.at[i, 'TeamName'] == 'North\xa0Ireland':
        lineups.at[i, 'TeamName'] = 'North Ireland'

    if(lineups.at[i, 'TeamName'] == 'L\'stein'):
        lineups.at[i, 'TeamName'] = 'Liechtenstein'

    if lineups.at[i, 'TeamName'] == 'San\xa0Marino':
        lineups.at[i, 'TeamName'] = 'San Marino'

    if lineups.at[i, 'TeamName'] == 'Moldova U21':
        lineups.at[i, 'TeamName'] = 'Moldova'

    if lineups.at[i, 'TeamName'] == 'England U21':
        lineups.at[i, 'TeamName'] = 'England'

### Adding country ID to Lineups

#### Senior games

In [13]:
for i in range(0, len(lineups)):
    if(pd.isna(lineups.at[i, 'TeamName'])):
        0==0
    else:
        this_country =  lineups.at[i, 'TeamName']
        if('U-20' in this_country):
            0==0
        else:
            
            code = countrycodes[countrycodes['Country'] == this_country].reset_index().at[0, ' Code']
                #print(i, this_country.split(' U-20')[0], code)
            lineups.at[i, ' Code'] = code
#lineups[' Code'] = lineups['TeamName'].apply(get_country_code)

#### U-20 games

In [14]:
#countrycodes[countrycodes['Country'] == this_country.split(' U-20')[0]].reset_index().at[0, ' Code']

for i in range(0, len(lineups)):
    if(pd.isna(lineups.at[i, 'TeamName'])):
        0==0
    else:
        this_country =  lineups.at[i, 'TeamName']
        if('U-20' in this_country):
            code = countrycodes[countrycodes['Country'] == this_country.split(' U-20')[0]].reset_index().at[0, ' Code']
            #print(i, this_country.split(' U-20')[0], code)
            lineups.at[i, ' Code'] = code

### Ad Hoc Fixing

In [15]:
for i in range(0, len(lineups)):
    if(lineups.at[i, 'TeamName'] == 'Chad'):
        lineups.at[i, ' Code'] = 'TD'
    if(lineups.at[i, 'TeamName'] == 'Namibia'):
        lineups.at[i, ' Code'] = 'NA'
    if(lineups.at[i, 'TeamName'] == 'Curacao'):
        lineups.at[i, ' Code'] = 'CW'

In [16]:
for i in range(0, len(lineups)):
    if(pd.isna(lineups.at[i, ' Code'])):
        print(i, lineups.at[i, 'TeamName'])
        0==0
    else:
        0==0
        #print(i, lineups.at[i, 'TeamName'])

#lineups = lineups.dropna(subset=[' Code']).reset_index()


## Adding Match ID to lineups

In [17]:
from datetime import datetime

def format_date(date_str):
    # Parse the input date string
    date_obj = datetime.strptime(date_str, '%d-%b-%y')

    # Check if the day has only one character, add leading zero if true
    day_str = date_obj.strftime('%d').rjust(2, '0')

    # Format the date string with the modified day
    formatted_date_str = date_obj.replace(day=int(day_str)).strftime('%d-%b-%y')

    return formatted_date_str

# Example usage:
input_date = '2-Feb-22'
formatted_date = format_date(input_date)
print(formatted_date)


02-Feb-22


In [18]:
def format_input_date_add_zeroes(input_date):
    # Split the input date into three sections
    sections = input_date.split('-')

    # Add leading zeros if necessary
    sections[0] = sections[0].rjust(2, '0')
    sections[1] = sections[1].rjust(2, '0')

    # Join the sections back together with '-'
    formatted_date = '-'.join(sections)

    return formatted_date

# Example usage:
input_date = '17-19-2021'
formatted_date = format_input_date_add_zeroes(input_date)
print(formatted_date)


17-19-2021


In [19]:
def has_letters(input_str):
    return any(char.isalpha() for char in input_str)

In [20]:
lineups['Found?'] = False
lineups['Match ID'] = 999999

In [21]:
correct_tally = 0
for i in range(0, len(lineups)):
#for i in range(4712, len(lineups)):
#for i in range(0, 4712):

    


#for i in range(5758, 5760):
#for i in range(5932, 5934):



    if(type(lineups.at[i, 'TeamName']) == float):
        0==0
    else:
    
        this_row_team = lineups.at[i, ' Code']
        this_row_comp = lineups.at[i, 'Competition']
        this_row_date = lineups.at[i, 'Date']

        row_match_results_1 = results[(results['Date'] == this_row_date) & (results['Team 1 Code'] == this_row_team)]
        row_match_results_2 = results[(results['Date'] == this_row_date) & (results['Team 2 Code'] == this_row_team)]
        row_match_results = pd.concat([row_match_results_1, row_match_results_2], ignore_index=True)

        if(len(row_match_results) == 1):
            correct_tally += 1
            correct_match_ID = row_match_results.at[0, 'Match ID']
            lineups.at[i, 'Match ID'] = correct_match_ID
        elif(len(row_match_results) >= 2):
            print('more than 1 results', i, this_row_team, lineups.at[i, 'Match'], lineups.at[i, 'Competition'], this_row_date)
        elif(len(row_match_results) == 0):

            if(has_letters(this_row_date)):
                formatted_date = format_date(this_row_date)

                row_match_results_1 = results[(results['Date'] == formatted_date) & (results['Team 1 Code'] == this_row_team)]
                row_match_results_2 = results[(results['Date'] == formatted_date) & (results['Team 2 Code'] == this_row_team)]
                new_df_datechange = pd.concat([row_match_results_1, row_match_results_2], ignore_index=True)

                if(len(new_df_datechange) == 1):
                    correct_tally += 1
                    correct_match_ID = new_df_datechange.at[0, 'Match ID']
                    lineups.at[i, 'Match ID'] = correct_match_ID
                else:
                    print("no rows month was a name",i, this_row_team, lineups.at[i, 'Match'], lineups.at[i, 'Competition'], formatted_date)
            
            else:
                formatted_date_plus_zeroes = format_input_date_add_zeroes(this_row_date)

                row_match_results_1 = results[(results['Date'] == formatted_date_plus_zeroes) & (results['Team 1 Code'] == this_row_team)]
                row_match_results_2 = results[(results['Date'] == formatted_date_plus_zeroes) & (results['Team 2 Code'] == this_row_team)]
                new_df_datechange = pd.concat([row_match_results_1, row_match_results_2], ignore_index=True)

                if(len(new_df_datechange) == 1):
                    correct_tally += 1
                    correct_match_ID = new_df_datechange.at[0, 'Match ID']
                    lineups.at[i, 'Match ID'] = correct_match_ID
                else:
                    print("no rows added zeroes",i, this_row_team, lineups.at[i, 'Match'], lineups.at[i, 'Competition'], formatted_date_plus_zeroes)



#match ID column 



more than 1 results 110 IT Ukraine U-20 vs Italy U-20 U-20 World Cup 2019 11.06.19
more than 1 results 114 UA Colombia U-20 vs Ukraine U-20 U-20 World Cup 2019 07.06.19
more than 1 results 1295 UA Ukraine vs Serbia Euros 2020, Qualifiers 07.06.19
more than 1 results 1705 IT Italy vs Bosnia-Herzegovina Euros 2020, Qualifiers 11.06.19
more than 1 results 4574 AF Afghanistan vs Syria World Cup Qualifiers, Asia, 2018 11-06-2015
more than 1 results 4575 SY Afghanistan vs Syria World Cup Qualifiers, Asia, 2018 11-06-2015
no rows added zeroes 4746 SA Saudi Arabia vs Palestine World Cup Qualifiers, Asia, 2018 11-06-2015
no rows added zeroes 4747 PS Saudi Arabia vs Palestine World Cup Qualifiers, Asia, 2018 11-06-2015
more than 1 results 4844 SA Saudi Arabia vs Palestine World Cup Qualifiers, Asia, 2018 11-06-2015
more than 1 results 4845 PS Saudi Arabia vs Palestine World Cup Qualifiers, Asia, 2018 11-06-2015
no rows added zeroes 4920 AF Afghanistan vs Syria World Cup Qualifiers, Asia, 2018 11

In [23]:
# rows_to_drop = [5856, 5857, 5932, 5933, 7150, 7151]

# lineups = lineups.drop(rows_to_drop, axis=0)

In [22]:
lineups[(lineups['Match ID'] == 999999)].head(60)

,TeamName,Starter_1,Starter_2,Starter_3,Starter_4,Starter_5,Starter_6,Starter_7,Starter_8,Starter_9,...,Unused_12,Unused_13,Unused_14,Unused_15,Competition,Date,Match,Code,Found?,Match ID
104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,na,False,999999
110,Italy U-20,1. Plizzari,14. Bellanova,15. Del Prato,3. Tripaldelli,13. Ranieri,17. Pellegrini Lu.,4. Gabbia,7. Frattesi,20. Esposito Sa.,...,-,-,-,-,U-20 World Cup 2019,11.06.19,Ukraine U-20 vs Italy U-20,IT,False,999999
114,Ukraine U-20,12. Kucheruk,2. Bondar V.,4. Popov D.,13. Beskorovaynyi,17. Konoplya,6. Chekh M.,9. Korniienko V.,10. Buletsa,15. Dryshlyuk,...,-,-,-,-,U-20 World Cup 2019,07.06.19,Colombia U-20 vs Ukraine U-20,UA,False,999999
209,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,na,False,999999
314,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,na,False,999999
443,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,na,False,999999
572,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,na,False,999999
701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,na,False,999999
1222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,na,False,999999
1295,Ukraine,12. Pyatov,16. Mykolenko,21. Karavayev,4. Kryvtsov,22. Matviyenko,6. Stepanenko,8. Malynovskyi,15. Tsygankov,17. Zinchenko,...,-,-,-,-,"Euros 2020, Qualifiers",07.06.19,Ukraine vs Serbia,UA,False,999999


In [23]:
lineups.at[110, 'Match ID'] = 54
lineups.at[114, 'Match ID'] = 56
lineups.at[1295, 'Match ID'] = 650
lineups.at[1705, 'Match ID'] = 855
# lineups.at[5586, 'Match ID'] = 2810
# lineups.at[5587, 'Match ID'] = 2810


# lineups.at[5758, 'Match ID'] = 2896
# lineups.at[5759, 'Match ID'] = 2896

# lineups.at[6968, 'Match ID'] = 3503
# lineups.at[6969, 'Match ID'] = 3503


In [29]:
#rows to delete in results
##row 2983 Afghanistan vs Syria 
##row 2945 Saudi Arabia vs Palestina
##row 3594 Israel vs North Macedonia
# rows_to_drop_results = [2983, 2945, 3594]

# results = results.drop(rows_to_drop_results, axis=0)


In [27]:
# lineups.at[6100, 'TeamName'] = "Iran"
# lineups.at[6100, ' Code'] = "IR"

# lineups.at[6100, 'Match ID'] = 3067

# lineups.at[6099, 'Match'] = "Cambodia vs Iran"
# lineups.at[6100, 'Match'] = "Cambodia vs Iran"



In [24]:
lineups[lineups['Match ID'] == 999999].tail(50)

,TeamName,Starter_1,Starter_2,Starter_3,Starter_4,Starter_5,Starter_6,Starter_7,Starter_8,Starter_9,...,Unused_12,Unused_13,Unused_14,Unused_15,Competition,Date,Match,Code,Found?,Match ID
2640,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,na,False,999999
2705,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,na,False,999999
2770,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,na,False,999999
3055,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,na,False,999999
3388,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,na,False,999999
3729,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,na,False,999999
3750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,na,False,999999
3811,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,na,False,999999
4062,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,na,False,999999
4377,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,na,False,999999


In [25]:
lineups.at[4574, 'Match ID'] = 2373
lineups.at[4574, 'Found?'] = True

lineups.at[4575, 'Match ID'] = 2373
lineups.at[4575, 'Found?'] = True

lineups.at[5956, 'Found?'] = True
lineups.at[5956, 'Match ID'] = 3084

lineups.at[5957, 'Found?'] = True
lineups.at[5957, 'Match ID'] = 3084

lineups.at[4746, 'Match ID'] = 2435
lineups.at[4746, 'Found?'] = True

lineups.at[4747, 'Match ID'] = 2435
lineups.at[4747, 'Found?'] = True

lineups.at[5088, 'TeamName'] = 'Iran'
lineups.at[5088, ' Code'] = 'IR'
lineups.at[5088, 'Match'] = 'Cambodia vs Iran'
lineups.at[5088, 'Match ID'] = 2557

lineups.at[5087, 'Match'] = 'Cambodia vs Iran'

# Define the indices of the rows you want to drop
indices_to_drop = [4844, 4845, 4920, 4921, 6138, 6139]

# Drop the rows
lineups = lineups.drop(indices_to_drop).reset_index().drop(columns= {'index'})


In [ ]:
#matches to look for in results 

#Afghanistan vs Syria 11-06-2015 - rows 4574, 4575
        #Match ID 2373 
        #Found? True

#Israel vs N Macedonia 02-09-2017 - rows 4746, 4747
        #Match ID 3084
        #FOund? True

#Saudi Arabia vs Palestine 11-6-2015
        #Match ID 2435
        #FOUND? True

#Rows to edit
#5088 - add teamName Iran, change game name to Cambodia vs Iran, add team code IR
#5087 - change game name to Cambodia vs Iran

#rows to delete
#4844, 4845 
#4920, 4921 
#6138, 6139 

In [26]:
lineups = lineups.dropna(subset=['TeamName'])


In [27]:
lineups[lineups['Match ID'] == 999999]

,TeamName,Starter_1,Starter_2,Starter_3,Starter_4,Starter_5,Starter_6,Starter_7,Starter_8,Starter_9,...,Unused_12,Unused_13,Unused_14,Unused_15,Competition,Date,Match,Code,Found?,Match ID
7653,Panama,-,-,-,-,-,-,-,-,-,...,-,-,-,-,"World Cup 2014, Qualifiers, CONCACAF",07-Oct-11,Bahamas vs Panama,PA,False,999999
7665,Panama,-,-,-,-,-,-,-,-,-,...,-,-,-,-,"World Cup 2014, Qualifiers, CONCACAF",07-Oct-11,Dominica vs Panama,PA,False,999999
7701,Nicaragua,-,-,-,-,-,-,-,-,-,...,-,-,-,-,"World Cup 2014, Qualifiers, CONCACAF",11-Nov-11,Bahamas vs Nicaragua,NI,False,999999
7712,Nicaragua,-,-,-,-,-,-,-,-,-,...,-,-,-,-,"World Cup 2014, Qualifiers, CONCACAF",11-Nov-11,Nicaragua vs Dominica,NI,False,999999


In [31]:
lineups[lineups['Date'] == '12-Nov-21']

,TeamName,Starter_1,Starter_2,Starter_3,Starter_4,Starter_5,Starter_6,Starter_7,Starter_8,Starter_9,...,Unused_12,Unused_13,Unused_14,Unused_15,Competition,Date,Match,Code,Found?,Match ID
8282,Canada,18 Borjan,22 Laryea,5 Vitória,4 Miller,3 Adekugbe,11 Buchanan,14 Kaye,7 Eustáquio,21 Millar,...,-,-,-,-,"World Cup 2022, Qualifiers, CONCACAF",12-Nov-21,Canada vs Costa Rica,CA,False,4167
8283,Costa Rica,23 Moreira,2 Blanco,6 Duarte,15 Calvo,8 Oviedo,17 Tejeda,12 Campbell,14 Galo,16 Salas,...,-,-,-,-,"World Cup 2022, Qualifiers, CONCACAF",12-Nov-21,Canada vs Costa Rica,CR,False,4167
8284,Honduras,22 López,2 Álvarez,16 Meléndez,3 Figueroa,21 Elvir,8 Mejia,10 López,20 Flores,7 Elis,...,-,-,-,-,"World Cup 2022, Qualifiers, CONCACAF",12-Nov-21,Honduras vs Panama,HN,False,4168
8285,Panama,1 Mejia,23 Murillo,4 Escobar,16 Andrade,15 Davis,6 Martinez,11 Cooper,8 Carrasquilla,19 Stephens,...,-,-,-,-,"World Cup 2022, Qualifiers, CONCACAF",12-Nov-21,Honduras vs Panama,PA,False,4168
8286,El Salvador,1 González,21 Tamacas,4 Zavaleta,5 R Gomez,13 Larín,6 Martínez,8 Landaverde,20 Dueñas,15 Roldan,...,-,-,-,-,"World Cup 2022, Qualifiers, CONCACAF",12-Nov-21,El Salvador vs Jamaica,SV,False,4169
8287,Jamaica,1 Blake,8 Fisher,17 Lowe,19 Mariappa,20 Lawrence,15 Watson,3 Grant,7 Bailey,16 Roofe,...,-,-,-,-,"World Cup 2022, Qualifiers, CONCACAF",12-Nov-21,El Salvador vs Jamaica,JM,False,4169
8288,USA,13 Steffen,2 Yedlin,12 Robinson,3 Zimmerman,5 Robinson,8 McKennie,4 Adams,6 Musah,20 Weah,...,-,-,-,-,"World Cup 2022, Qualifiers, CONCACAF",12-Nov-21,USA vs Mexico,US,False,4170
8289,Mexico,13 Ochoa,21 Rodríguez,3 Domínguez,5 Vásquez,23 Gallardo,16 Herrera,4 Álvarez,7 Romo,17 Corona,...,-,-,-,-,"World Cup 2022, Qualifiers, CONCACAF",12-Nov-21,USA vs Mexico,MX,False,4170
8799,Uruguay,1 Muslera,22 Cáceres,2 Giménez,3 Godín,17 Piquerez,14 Torreira,8 Nández,5 Vecino,6 Bentancur,...,-,-,-,-,"World Cup 2022, Qualifiers, CONMEBOL",12-Nov-21,Uruguay vs Argentina,UY,False,4427
8800,Argentina,23 Martínez,5 Molina,13 Romero,19 Otamendi,8 Acuña,7 d Paul,18 Rodríguez,11 D Maria,21 Dybala,...,-,-,-,-,"World Cup 2022, Qualifiers, CONMEBOL",12-Nov-21,Uruguay vs Argentina,AR,False,4427


# EXPORTING DATA

In [40]:
results.to_csv('newest_results_data_march_24.csv', index=False)


In [41]:
lineups.to_csv('newest_lineups_data_march_24.csv', index=False)
